In [ ]:
import simpy
import random
from statistics import mean
import matplotlib.pyplot as plt
import numpy as np
import csv
import pandas as pd

random.seed(161)
# Arrivals generator function
def patient_generator(env, p_inter, reg_time, gp_time, bt_time, test_prob, receptionist, gp):
    p_id = 0
    
    # Keep generating indefinitely
    while True:
        #Create instance of activity generator for registration
        pa = activity_generator_registration(env, reg_time, gp_time, bt_time, test_prob, receptionist, gp, p_id)
        
        #Run the activity generator for registration
        env.process(pa)
        
        # Establish interarrival time
        t = random.expovariate(1.0/p_inter)
        
        #Freeze until that time has passed
        yield env.timeout(t)
        
        p_id +=1
        
def activity_generator_registration(env, reg_time, gp_time, bt_time, test_prob, receptionist, gp, p_id):
    global list_of_reg_queue_times
    global receptionist_busy_time
    time_entered_queue_for_registration = env.now
   
    #Request a receptionist
    with receptionist.request(priority = 2) as req:
        # Freeze until the request can be met
        yield req
        
        time_left_queue_for_registration = env.now
        time_in_queue_for_registration = (time_left_queue_for_registration -time_entered_queue_for_registration)
        list_of_reg_queue_times.append(time_in_queue_for_registration)
        #print("Patient ", p_id, " queued for Registration for ", np.round(time_in_queue_for_registration,2)," minutes", sep='')
        
        
        #Sample time spent with receptionist
        registration_time = random.expovariate(1.0/reg_time)
        
        #Freeze until that time has passed
        yield env.timeout(registration_time)
        receptionist_busy_time += registration_time
        
    #Create instance of activity generator for gp consultation
    gpa = activity_generator_gp_consultation(env, gp_time, bt_time, test_prob, receptionist, gp, p_id)
        
    #Run the activity generator for GP Consultation
    env.process(gpa)
    
def activity_generator_gp_consultation(env, gp_time, bt_time, test_prob, receptionist, gp, p_id):
    global list_of_gp_consult_queue_times
    global gp_busy_time    
    time_entered_queue_for_gp_consult = env.now
   
    #Request a GP
    with gp.request() as req:
        # Freeze until the request can be met
        yield req
        
        time_left_queue_for_gp_consult = env.now
        time_in_queue_for_gp_consult = (time_left_queue_for_gp_consult -time_entered_queue_for_gp_consult)
        list_of_gp_consult_queue_times.append(time_in_queue_for_gp_consult)
        #print("Patient ", p_id, " queued for GP Consultation for ", np.round(time_in_queue_for_gp_consult,2)," minutes", sep='')
        
        
        #Sample time spent with gp
        gp_consult_time = random.expovariate(1.0/gp_time)
        
        #Freeze until that time has passed
        yield env.timeout(gp_consult_time)
        gp_busy_time += gp_consult_time

    
    random_prob = random.uniform(0,1)
    
    # Determine if need to book test
    if random_prob < test_prob:
        bpa = activity_generator_book_test(env,receptionist, bt_time, p_id)
        
        #Run the activity generator for booking a test
        env.process(bpa)
        
def activity_generator_book_test(env, receptionist, bt_time, p_id):
    global list_of_book_test_queue_times
    global receptionist_busy_time    
    time_entered_queue_for_book_test = env.now
   
    #Request a GP
    with receptionist.request(priority = 1) as req:
        # Freeze until the request can be met
        yield req
        
        time_left_queue_for_book_test = env.now
        time_in_queue_for_book_test = (time_left_queue_for_book_test -time_entered_queue_for_book_test)
        list_of_book_test_queue_times.append(time_in_queue_for_book_test)
        #print("Patient ", p_id, " queued to Book Test for ", np.round(time_in_queue_for_book_test,2)," minutes", sep='')
        
        
        #Sample time spent booking
        book_time = random.expovariate(1.0/bt_time)
        
        #Freeze until that time has passed
        yield env.timeout(book_time)
        receptionist_busy_time += book_time

        
#Create a file to store the results of each run, and write the column headers
with open("gp_clinic.csv", "w") as f:
    writer = csv.writer(f, delimiter = ",")
    writer.writerow(["Run", "Mean Registration Queue", "Mean GP Consult Queue", "Mean Book Test Queue", 
                     "Mean Receptionist Utilization", "Mean GP Utilization"])

# Identify number of simulations
num_sims = 100

for run in range (num_sims):
    # Set up simulation environment
    env = simpy.Environment()

    #Set up resources
    receptionist = simpy.PriorityResource(env, capacity = 1)
    gp = simpy.Resource(env, capacity = 2)

    #Set up parameter values
    p_inter = 3
    reg_time = 2
    gp_time = 8
    bt_time = 4
    test_prob = 0.25
    SimTime = 8*60

    #Set up lists for global variables
    list_of_reg_queue_times = []
    list_of_gp_consult_queue_times = []
    list_of_book_test_queue_times = []
    receptionist_busy_time = 0
    gp_busy_time = 0

    # Start the arrivals generator
    env.process(patient_generator(env, p_inter, reg_time, gp_time, bt_time, test_prob, receptionist, gp))


    #Run the simulation for 8 hours 
    env.run(until=SimTime)

    mean_queue_time_registration = np.mean(list_of_reg_queue_times)
    mean_queue_time_gp_consult = np.mean(list_of_gp_consult_queue_times)
    mean_queue_time_book_test = np.mean(list_of_book_test_queue_times)
    mean_receptionist_utilization = receptionist_busy_time/(SimTime)
    mean_gp_utilization = gp_busy_time/(SimTime*2)
    #print('The mean time waiting for registration was ', np.round(mean_queue_time_registration,2))
    #print('The mean time waiting for GP consultation was ', np.round(mean_queue_time_gp_consult,2))
    #print('The mean time waiting to book test was ', np.round(mean_queue_time_book_test,2))
    #print('The average utilization of the receptionist was ', np.round(mean_receptionist_utilization,2))
    #print('The average utilization of the GP was ', np.round(mean_gp_utilization,2))
    
    list_to_write = [run, mean_queue_time_registration, mean_queue_time_gp_consult, mean_queue_time_book_test, mean_receptionist_utilization, mean_gp_utilization]
    
    # Write result of each replication to file
    with open("gp_clinic.csv", "a") as f:
        writer = csv.writer(f, delimiter = ",")
        writer.writerow(list_to_write)
        
#Here we are going to read this into a pandas dataframe that allows us to easily
# access the results using the column names given above.
results_df = pd.read_csv("gp_clinic.csv")
mean_trial_queuing_time_registration = results_df["Mean Registration Queue"].mean()
print("Mean queuing time for Registration was ", mean_trial_queuing_time_registration)

plt.figure()
plt.hist(results_df["Mean Registration Queue"],density=True,histtype='stepfilled')
plt.show()

mean_trial_queuing_time_gp_consult = results_df["Mean GP Consult Queue"].mean()
print("Mean queuing time for GP Consult was ", mean_trial_queuing_time_gp_consult)

plt.figure()
plt.hist(results_df["Mean GP Consult Queue"],density=True,histtype='stepfilled')
plt.show()

mean_trial_queuing_time_book_test = results_df["Mean Book Test Queue"].mean()
print("Mean queuing time for Book Test was ", mean_trial_queuing_time_book_test)

plt.figure()
plt.hist(results_df["Mean Book Test Queue"],density=True,histtype='stepfilled')
plt.show()

mean_utilization_receptionist = results_df["Mean Receptionist Utilization"].mean()
print("Mean utilization for the Receptionist was ", mean_utilization_receptionist)

plt.figure()
plt.hist(results_df["Mean Receptionist Utilization"],density=True,histtype='stepfilled')
plt.show()

mean_utilization_gp = results_df["Mean GP Utilization"].mean()
print("Mean utilization for the GP was ", mean_utilization_gp)

plt.figure()
plt.hist(results_df["Mean GP Utilization"],density=True,histtype='stepfilled')
plt.show()
